In [2]:
import numpy as np 
import pandas as pd 
import os
import csv
import warnings
warnings.simplefilter(action='ignore')
import gc
from pathlib import Path 


In [28]:
df_combined = pd.read_csv('sampled_dataset_full_column.csv')

columns = ["userid","username","location","tweetid","tweetcreatedts","retweetcount", "hashtags", "text", "usercreatedts", "following", "followers","totaltweets"]

# select equal timeframe bafore and after the ban of twitter in Russia
df1 = df_combined[columns][df_combined['tweetcreatedts'] > "2022-02-20 00:00:00" ] 
df1 = df1[df1['tweetcreatedts'] < "2022-03-14 00:00:00" ] 
df1

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets
408402,879477038599487488,felionaaaa,NaN,1.497216e+18,2022-02-25 14:24:32,261.0,"[{'text': 'Ukraine', 'indices': [67, 75]}, {'t...",President of ukraine is leading from the front...,2017-06-26 23:10:40.000000,117.0,15.0,4482.0
408403,910979270054338561,Patrici25530591,NaN,1.497175e+18,2022-02-25 11:40:25,41635.0,[],.@ZelenskyyUa's tv address to the Russian (!) ...,2017-09-21 21:29:18.000000,61.0,32.0,3959.0
408404,983264761,eantonicelli,"Stavanger, Norway",1.497525e+18,2022-02-26 10:50:58,150.0,[],The Ukrainian civilians who were handed out 18...,2012-12-01 20:15:21.000000,2880.0,2590.0,19074.0
408405,1144411063984828418,YoDammit,VA,1.497778e+18,2022-02-27 03:38:45,3396.0,"[{'text': 'Russia', 'indices': [59, 66]}]",⚡ Intel and AMD stop supplying microchips to #...,2019-06-28 01:03:54.000000,79.0,33.0,5521.0
408406,2369760817,CharleyBurt1,USA,1.497116e+18,2022-02-25 07:46:59,563.0,[],"I’m Russian, and I want you to know that the R...",2014-03-03 03:13:04.000000,3351.0,4112.0,46789.0
...,...,...,...,...,...,...,...,...,...,...,...,...
461555,871858796179673089,ThatVDOVault,Shenandoah Valley VA USA,1.502972e+18,2022-03-13 11:36:03,224.0,"[{'text': 'StandWithUkraine', 'indices': [72, ...","Andriy Shevchenko: please, save Ukrainian chil...",2017-06-05 22:38:30.000000,3383.0,2645.0,302920.0
461556,1490737887658360833,RedwanAziz99,"Strasbourg, France",1.503080e+18,2022-03-13 18:47:46,54.0,"[{'text': 'Ukraine', 'indices': [54, 62]}]",The world's response to the war in #Ukraine ha...,2022-02-07 17:23:21.000000,207.0,487.0,12957.0
461557,761078091745599488,IAmACatJustCat,NaN,1.503103e+18,2022-03-13 20:17:29,0.0,"[{'text': 'StopRussianAggression', 'indices': ...","@airserbia While your planes fly to russia, ru...",2016-08-04 05:55:32.000000,245.0,109.0,6798.0
461558,1498588978173280268,bigwincam,NaN,1.503037e+18,2022-03-13 15:56:34,207.0,"[{'text': 'FREEDOM', 'indices': [67, 75]}, {'t...","IF YOU PERSIST, YOU WILL PREVAIL 🙏\n\nFIGHT FO...",2022-03-01 09:20:44.000000,561.0,116.0,954.0


### Hashtags

### (1) top used hashtags 

In [16]:
tags = df1['hashtags'].tolist()

TAGS=[]
for item in tags:
    TAGS+=item[1:-1].split('{')
TAGS2=[]
for item2 in TAGS:
    if item2!='':
        TAGS2+=[item2.split(': ')[1].split(',')[0][1:-1]]
tagdf=pd.DataFrame(pd.Series(TAGS2).value_counts())
tagdf['tag']=tagdf.index.tolist()
tagdf.columns=['counts','tag']
tagdf.index=range(len(tagdf))

tag_c = tagdf[tagdf['counts']>30]
tag_c.to_csv("hashtag_count_over30.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

### (2) clean hashtags for each tweet 

In [29]:
# write a function to extract "hashtags" 
def SplitTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

def LowerTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    lower_TAGS = (map(lambda x: x.lower(), TAGS))
    TAGS = list(lower_TAGS)
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

            
df1['tags_splitted'] = df1['hashtags'].apply(SplitTags)
df1['tags_lower'] = df1['hashtags'].apply(LowerTags)
df1

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower
408402,879477038599487488,felionaaaa,NaN,1.497216e+18,2022-02-25 14:24:32,261.0,"[{'text': 'Ukraine', 'indices': [67, 75]}, {'t...",President of ukraine is leading from the front...,2017-06-26 23:10:40.000000,117.0,15.0,4482.0,"[Ukraine, UkraineRussia, Putin, UkraineInvasio...","[ukraine, ukrainerussia, putin, ukraineinvasio..."
408403,910979270054338561,Patrici25530591,NaN,1.497175e+18,2022-02-25 11:40:25,41635.0,[],.@ZelenskyyUa's tv address to the Russian (!) ...,2017-09-21 21:29:18.000000,61.0,32.0,3959.0,NA,NA
408404,983264761,eantonicelli,"Stavanger, Norway",1.497525e+18,2022-02-26 10:50:58,150.0,[],The Ukrainian civilians who were handed out 18...,2012-12-01 20:15:21.000000,2880.0,2590.0,19074.0,NA,NA
408405,1144411063984828418,YoDammit,VA,1.497778e+18,2022-02-27 03:38:45,3396.0,"[{'text': 'Russia', 'indices': [59, 66]}]",⚡ Intel and AMD stop supplying microchips to #...,2019-06-28 01:03:54.000000,79.0,33.0,5521.0,[Russia],[russia]
408406,2369760817,CharleyBurt1,USA,1.497116e+18,2022-02-25 07:46:59,563.0,[],"I’m Russian, and I want you to know that the R...",2014-03-03 03:13:04.000000,3351.0,4112.0,46789.0,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461555,871858796179673089,ThatVDOVault,Shenandoah Valley VA USA,1.502972e+18,2022-03-13 11:36:03,224.0,"[{'text': 'StandWithUkraine', 'indices': [72, ...","Andriy Shevchenko: please, save Ukrainian chil...",2017-06-05 22:38:30.000000,3383.0,2645.0,302920.0,"[StandWithUkraine, PutinIsaWarCriminal, PutinH...","[standwithukraine, putinisawarcriminal, putinh..."
461556,1490737887658360833,RedwanAziz99,"Strasbourg, France",1.503080e+18,2022-03-13 18:47:46,54.0,"[{'text': 'Ukraine', 'indices': [54, 62]}]",The world's response to the war in #Ukraine ha...,2022-02-07 17:23:21.000000,207.0,487.0,12957.0,[Ukraine],[ukraine]
461557,761078091745599488,IAmACatJustCat,NaN,1.503103e+18,2022-03-13 20:17:29,0.0,"[{'text': 'StopRussianAggression', 'indices': ...","@airserbia While your planes fly to russia, ru...",2016-08-04 05:55:32.000000,245.0,109.0,6798.0,"[StopRussianAggression, RussianUkrainianWar]","[stoprussianaggression, russianukrainianwar]"
461558,1498588978173280268,bigwincam,NaN,1.503037e+18,2022-03-13 15:56:34,207.0,"[{'text': 'FREEDOM', 'indices': [67, 75]}, {'t...","IF YOU PERSIST, YOU WILL PREVAIL 🙏\n\nFIGHT FO...",2022-03-01 09:20:44.000000,561.0,116.0,954.0,"[FREEDOM, stopwar, UKRAINE]","[freedom, stopwar, ukraine]"


### (3) create node attribute of political orientation

In [26]:
# import tashtag file 
df_tags = pd.read_csv('hashtag_over30_tagged.csv')

pro_R_tags = df_tags[df_tags["Pro_R"]=="TRUE"]['tag'].to_list()

pd.crosstab(index=df_tags['Pro_R'], columns='count') # 5
pd.crosstab(index=df_tags['Pro_U'], columns='count') #61
pd.crosstab(index=df_tags['Neutral'], columns='count') #178

pro_R_tags

['RussiaUkraineConflict',
 'IStandWithPutin',
 'OpRussia',
 'UkraineConflict',
 'istandwithrussia']

In [30]:
pro_R_tags = ["IStandWithPutin","istandwithrussia","ISupportRussia","ZelenskyWarCriminal","Zenlenskyisawarcriminal",\
              "NaziUkraine","UkraineNazis","nazisinukraine","WarCrimesOfUkraine","PutinIsRight","StopHatingRussians",\
              "NATOWarCriminal", "PatrioticRussia"]
pro_U_tags = ["Istandwithukraine","Putinisawarcriminal","PutinWarCriminal","StandWithUkraine", "HandsOffUkraine",\
              "kievfree","UkraineUnderAttack","StopRussia","StopPutin","RussiaInvadedUkriane","StopPutinsWar",\
              "StopRussianAggression","ukraineinvasion","russianinvasion","RussiaInvadedUkraine","PrayingForUkraine",\
              "RussiaOutOfUkraine","StandWithUkriane","RussiaInvadesUkraine","PutinHitler"]

# change all tags to lowercase before matching to avoid spelling difference
lower_R = (map(lambda x: x.lower(), pro_R_tags))
pro_R_tags = list(lower_R)

lower_U = (map(lambda x: x.lower(), pro_U_tags))
pro_U_tags = list(lower_U)

# lambda functions marking political orientation if related hashtags were found 
pro_R_check = lambda taglist: bool(set(taglist) & set(pro_R_tags)) # return true if overlap of two sets not empty
pro_U_check = lambda taglist: bool(set(taglist) & set(pro_U_tags)) 

df1['pro_Russian'] = df1['tags_lower'].apply(pro_R_check)
df1['pro_Ukraine'] = df1['tags_lower'].apply(pro_U_check)

In [17]:
no_tag = df1[df1['tags_splitted']=="NA"]
# 10594 tweets with no hashtags

# 255 labeled as pro-Russia
df1[df1['pro_Russian']==True]

# 6412 labeled as pro-Ukraine
df1[df1['pro_Ukraine']==True]

# still a lot rabeled by neither of two groups
non_R = df1[df1['pro_Russian']==False]
non = non_R[non_R['pro_Ukraine']==False]
non


,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower,pro_Russian,pro_Ukraine
408403,910979270054338561,Patrici25530591,NaN,1.497175e+18,2022-02-25 11:40:25,41635.0,[],.@ZelenskyyUa's tv address to the Russian (!) ...,2017-09-21 21:29:18.000000,61.0,32.0,3959.0,NA,NA,False,False
408404,983264761,eantonicelli,"Stavanger, Norway",1.497525e+18,2022-02-26 10:50:58,150.0,[],The Ukrainian civilians who were handed out 18...,2012-12-01 20:15:21.000000,2880.0,2590.0,19074.0,NA,NA,False,False
408405,1144411063984828418,YoDammit,VA,1.497778e+18,2022-02-27 03:38:45,3396.0,"[{'text': 'Russia', 'indices': [59, 66]}]",⚡ Intel and AMD stop supplying microchips to #...,2019-06-28 01:03:54.000000,79.0,33.0,5521.0,[Russia],[russia],False,False
408406,2369760817,CharleyBurt1,USA,1.497116e+18,2022-02-25 07:46:59,563.0,[],"I’m Russian, and I want you to know that the R...",2014-03-03 03:13:04.000000,3351.0,4112.0,46789.0,NA,NA,False,False
408407,170191353,SpeachTherapy,"MiAMi, i'll SOUTH BEACH ya",1.497089e+18,2022-02-25 06:01:12,2333.0,"[{'text': 'Ukraine', 'indices': [28, 36]}, {'t...",Terrifying. #Ukraine #Russia https://t.co/VF6S...,2010-07-24 04:38:19.000000,255.0,159.0,7160.0,"[Ukraine, Russia]","[ukraine, russia]",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461552,481389462,edwards_juli,Wales,1.503080e+18,2022-03-13 18:47:30,404.0,"[{'text': 'UkraineWar', 'indices': [19, 30]}]",#UkraineWar Ukrainian forces just captured a R...,2012-02-02 17:54:38.000000,378.0,187.0,6289.0,[UkraineWar],[ukrainewar],False,False
461553,1118455785162297344,ohashi_dan,North caloocan,1.503046e+18,2022-03-13 16:30:53,0.0,[],this would be the last time!,2019-04-17 10:06:53.000000,106.0,211.0,1421.0,NA,NA,False,False
461554,1036991116963246080,reicocurori169,NaN,1.502911e+18,2022-03-13 07:33:28,2821.0,"[{'text': 'russiansoldiers', 'indices': [19, 3...",#russiansoldiers shot an old #Ukrainian couple...,2018-09-04 14:55:23.000000,2745.0,18.0,41631.0,"[russiansoldiers, Ukrainian]","[russiansoldiers, ukrainian]",False,False
461556,1490737887658360833,RedwanAziz99,"Strasbourg, France",1.503080e+18,2022-03-13 18:47:46,54.0,"[{'text': 'Ukraine', 'indices': [54, 62]}]",The world's response to the war in #Ukraine ha...,2022-02-07 17:23:21.000000,207.0,487.0,12957.0,[Ukraine],[ukraine],False,False


In [20]:
df_R = non[non['text'].str.contains('Zenlensky' and 'Greenscreen')]
# tried keywords “Zenlensky” AND “Greenscreen” but no results in the non-labeled 

In [21]:
df_R

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower,pro_Russian,pro_Ukraine


### Nodelist with political stance

In [59]:
node_columns = ["userid","username","usercreatedts","following","followers","totaltweets","pro_Russian","pro_Ukraine"]
nodelist1 = df1[node_columns][(df1['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
nodelist2 = df1[node_columns][(df1['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
# code for nodetype variable

def Poli_Stance(Pro_R, Pro_U):
    if df1[Pro_R] == "True":
        return "R"
    elif df1[Pro_U] == "True":
        return "U"
    else:
        return "NA"
    
#nodelist["NodeType"] = Poli_Stance(df1["pro_Russian"],df1["pro_Ukraine"])

nodelist1['NodeType'] = np.where(
     nodelist1['pro_Russian'], 'pro_Russian', 
     np.where(
        nodelist1['pro_Ukraine'], 'pro_Ukraine', 'Unknown'
     )
)

nodelist2['NodeType'] = np.where(
     nodelist2['pro_Russian'], 'pro_Russian', 
     np.where(
        nodelist2['pro_Ukraine'], 'pro_Ukraine', 'Unknown'
     )
)

In [65]:



nodelist1
nodelist2

#pd.crosstab(index=nodelist['pro_Russian'], columns='count')
pd.crosstab(index=nodelist2['NodeType'], columns='count') #178

col_0,count
NodeType,
Unknown,26255
pro_Russian,86
pro_Ukraine,3661


In [66]:
nodelist1.to_csv("nodelist1_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
nodelist2.to_csv("nodelist2_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

### Edgelist for user->hashtag mapping

In [64]:
hashtags = df1['tags_splitted'][0]


['Ukraine', 'UkraineRussia', 'Putin', 'UkraineInvasion', 'Ukrainian', 'แตงโม']

In [ ]:
def UHmapping(hashtags):
    TAGS=[]
    if hashtags != 'NA':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]

edgelist['hashtag'] = df1['hashtags'].apply(UHmapping)
                          

                          

In [74]:
edgelist = {'username':[],'tag':[]}
#df1 = df1.reset_index()  # make sure indexes pair with number of rows
for index, row in df1.iterrows():
    user = row['username']
    hashtags = row['tags_splitted']
    # change pandas series to list
    if hashtags != "NA":
        hashtags = list(pd.Series(hashtags).values)
        for tag in hashtags:
            edgelist.setdefault('username',[]).append(user)
            edgelist.setdefault('tag',[]).append(tag) 

edgelist = pd.DataFrame.from_dict(edgelist)

In [77]:
edgelist

,username,tag
0,felionaaaa,Ukraine
1,felionaaaa,UkraineRussia
2,felionaaaa,Putin
3,felionaaaa,UkraineInvasion
4,felionaaaa,Ukrainian
...,...,...
108769,bigwincam,UKRAINE
108770,liontwee,Putin
108771,liontwee,StopPutinsWar
108772,liontwee,PutinIsaWarCriminal


In [78]:
edgelist.to_csv("edgelist_user_tag.csv",sep = ",", header = True, encoding = "UTF-8",index=False)